#  Capstone Project

## Introduction/Business Problem  

### Objective:
The idea of this machine learning project is to build a similar neighbourhood recommendation system.
### Target Audience:
The Business Problem is related to many People who do often switch companies for a better opportunity. But while switching, most of the times people have to change the current Neighbourhood/City/Country. They always wish that they get all same required things like services, enjoyments, clubs, restaurants, hangout places etc in the new Neighbourhood/City/Country. So Is there a way we can recommend them best neighbourhoods near their new office.
### Introduction / Business Problem :
So Here in this project, we are going to recommend the best and same type of neighbourhoods as their current neighbourhood to a user in terms of service, search for the potential explanation of why a neighbourhood is popular, the cause of complaints in another neighbourhood, or anything else related to neighbourhoods.

   #### Success criteria of the project are :
     - define common cluster/class values for similar neighborhoods in London / New York
     - deliver optimized model for these classes
     - provide a list of similar neighborhoods within the chosen cities
     - show the recommended neighborhood on a map 


## Data Gathering, Cleansing and Exploratory Data Analysis

### Importing libs 

In [295]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from bs4 import BeautifulSoup # library of Beautifulsoup

from folium.plugins import MarkerCluster 
import folium  # plotting library
 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import pgeocode
nomi = pgeocode.Nominatim('CA')


# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
pd.set_option('display.max_columns', None)

print('Folium installed')
print('Libraries imported.')


colors = [
    'red',
    'blue',
    'gray',
    'darkred',
    'lightred',
    'orange',
    'beige',
    'green',
    'darkgreen',
    'lightgreen',
    'darkblue',
    'lightblue',
    'purple',
    'darkpurple',
    'pink',
    'cadetblue',
    'lightgray',
    'black'
]


CLIENT_ID = 'WJALSUPQARDIIVEU4NV1RWEEFGT0DZNNX0KQTVCSX5LZNIGI' # your Foursquare ID
CLIENT_SECRET = 'RUZPK1EYFBKDLKNTDF1QKHBOMGYWG3C0JICCP0Y1C5S2LCPK' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
radius = 500

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Folium installed
Libraries imported.
Your credentails:
CLIENT_ID: WJALSUPQARDIIVEU4NV1RWEEFGT0DZNNX0KQTVCSX5LZNIGI
CLIENT_SECRET:RUZPK1EYFBKDLKNTDF1QKHBOMGYWG3C0JICCP0Y1C5S2LCPK


### Getting Wikipage Table Data (Web Scraping part)

In [2]:
# getting html text

wikipage = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

soup = BeautifulSoup(wikipage.text,"html.parser")

# finding all texts which contains <table> tag
tables = soup.find_all("table")

# need first table only
final_table = tables[0]

# getting table data
table_data = [[cell.text for cell in row("td")]
                         for row in final_table("tr")]
#getting table headers
table_columns = [x.text for x in final_table("th")]

###  Cleaning and formating cluster Dataframe

In [3]:
# Creating cluster dataframe using table data and table columns and removing '\n'
cluster_df = pd.DataFrame(table_data[1:],columns=table_columns).rename(columns={"Neighbourhood\n":"Neighbourhood"})

cluster_df.loc[:,"Neighbourhood"] = cluster_df["Neighbourhood"].str.replace('\n','').values

# Cleaning data
cluster_assigned_df = cluster_df.loc[cluster_df["Borough"]!="Not assigned",:].reset_index(drop=True)

# Fromating naeighbours into the list
cluster_assigned_df = cluster_assigned_df.groupby(["Postcode","Borough"],as_index=True).apply(lambda x : ','.join(x["Neighbourhood"].tolist()))

cluster_assigned_df = cluster_assigned_df.reset_index().rename(columns={0:"Neighbourhood"})

# if neighbourhood is 'Not Assigned' then assigning 'Borough' name there
cluster_assigned_df.loc[:,"Neighbourhood"] = cluster_assigned_df.T.apply(lambda x : x["Borough"] if x["Neighbourhood"]=="Not assigned" else x["Neighbourhood"]).values

####  Total Rows and Columns 

In [4]:
print("Rows : {}  Columns: {}".format(cluster_assigned_df.shape[0],cluster_assigned_df.shape[1]))

Rows : 103  Columns: 3


### Getting Latitude and Longitude and merging with cluster dataframe 

In [77]:
# getting lats and long from given method so using this file

def get_long_lat(postcode):
    """ Method takes a Series object and returns
    a list of Latitude and corresponding Longitude data,
    using the pgeocode library.
    This method also prints out the coordinate data"""
    
    if postcode=="M7R":
        latitude = 43.637
        longitude = -79.616
    
    else:
        location = nomi.query_postal_code(postcode)
        latitude = location.latitude
        longitude = location.longitude
    return [latitude, longitude]

In [79]:
latt_longs = clusters_df["Postcode"].apply(get_long_lat)

clusters_df.loc[:,"Latitude"] = latt_longs.apply(lambda col: col[0])
clusters_df.loc[:,"Longitude"] = latt_longs.apply(lambda col: col[1])

# Showing first 5 rows 

clusters_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.8113,-79.1930
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.7878,-79.1564
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.7678,-79.1866
3,M1G,Scarborough,Woburn,43.7712,-79.2144
4,M1H,Scarborough,Cedarbrae,43.7686,-79.2389


### visualizing clusters

In [81]:


clr_i = 0

some_map = folium.Map(location=(43.65,-79.38) , zoom_start=11)

for city in clusters_df.Borough.unique():
    city_neighbs = clusters_df.loc[clusters_df.Borough == city,['Borough','Neighbourhood',"Postcode",'Latitude', 'Longitude']]
    neighs = folium.map.FeatureGroup()
    clr = colors[clr_i]
    clr_i+=1
    for br,nm,pc,lat,lng in city_neighbs.values:
        folium.CircleMarker(
            [lat, lng],
            radius=3, 
            color=clr,
            fill=True,
            popup= "<br>Borough ==> {} And <br>Neighbourhood ==> {}".format(br,nm),
            fill_opacity=0.8
        ).add_to(some_map)

some_map

### Let's Pick one of the PostCodes and explore the all venues 

In [181]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [184]:
# search_query = borough_df["Borough"]
# search_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
#     CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

In [305]:
def explore_venues(neigbh , latitude , longitude):
    explore_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET,  VERSION, latitude, longitude,  radius,  LIMIT)

    result = requests.get(explore_url).json()
    
#     print(result.keys())
    venues = result["response"]["groups"][0]["items"]
    
    
#     print(len(venues))
    
    if len(venues)==0:
        return pd.DataFrame()
    
    nearby_venues = json_normalize(venues)

    # # filter columns
    filtered_columns = ['venue.name', 'venue.categories','venue.location.city', 'venue.location.lat', 'venue.location.lng', 'venue.location.distance']
    nearby_venues =nearby_venues.loc[:, filtered_columns]

    # filter the category for each row
    nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
    nearby_venues["venue.location.city"] = nearby_venues["venue.location.city"].fillna(method = "ffill")

    # # clean columns
    nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
    
    nearby_venues.loc[:,"Neighbourhood"] = neigbh
    nearby_venues.loc[:,"latitude"] = latitude
    nearby_venues.loc[:,"longitude"] = longitude

    return nearby_venues

In [298]:
west_toronto_venues = pd.DataFrame()
for neigbh,latitude,longitude  in  clusters_df.loc[clusters_df["Borough"] == "West Toronto",["Neighbourhood","Latitude","Longitude"]].values:
    print(neigbh,latitude,longitude )
    venue_df = explore_venues(neigbh , latitude , longitude)
    
    west_toronto_venues = pd.concat([west_toronto_venues,venue_df],sort=False)

dict_keys(['meta', 'response'])
20
dict_keys(['meta', 'response'])
65
dict_keys(['meta', 'response'])
38
dict_keys(['meta', 'response'])
3
dict_keys(['meta', 'response'])
46
dict_keys(['meta', 'response'])
49


### Top 5 most common Venues

In [374]:
west_toronto_categories = pd.get_dummies(west_toronto_venues["categories"], prefix="", prefix_sep="")
west_toronto_categories.loc[:,"Neighbourhood"] = west_toronto_venues["Neighbours"].values
west_toronto_venues_grouped = west_toronto_categories.groupby("Neighbourhood").mean()

west_toronto_venues_grouped.T.apply(lambda x : x.sort_values(ascending=False).index[:5])

Neighbourhood,"Brockton,Exhibition Place,Parkdale Village","Dovercourt Village,Dufferin","High Park,The Junction South","Little Portugal,Trinity","Parkdale,Roncesvalles","Runnymede,Swansea"
0,Coffee Shop,Park,Park,Bar,Eastern European Restaurant,Café
1,Breakfast Spot,Furniture / Home Store,Intersection,Asian Restaurant,Coffee Shop,Coffee Shop
2,Thrift / Vintage Store,Bakery,Pub,Coffee Shop,Bakery,Pizza Place
3,Café,Smoke Shop,Fish & Chips Shop,Restaurant,Sushi Restaurant,Gastropub
4,Lounge,Gym / Fitness Center,Cuban Restaurant,Pizza Place,Breakfast Spot,Restaurant
